# 디렉토리 설정 및 이미지데이터 전처리

In [1]:
import glob
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd drive/MyDrive/CNN_upgrade/

/content/drive/MyDrive/CNN_upgrade


In [5]:
!pwd

/content/drive/MyDrive/CNN_upgrade


In [6]:
base_dir = './'
categories = ['datasets/pedestrian', 'datasets/sitter', 'datasets/taxier']
nb_classes = len(categories)

In [7]:
# 이미지 크기 지정
image_w = 256
image_h = 256
pixels = image_w*image_h*3

In [8]:
# 이미지데이터 읽어들이기
X = []
Y = []
for idx, category in enumerate(categories):
    # 레이블 지정
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    
    # 이미지
    image_dir = base_dir+category+'/*'
    files = glob.glob(image_dir)
    
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
    
X = np.array(X)
Y = np.array(Y)

# 학습 전용 데이터와 테스트 전용 데이터로 구분
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
# 데이터 정규화하기(0~1사이로)
X_train = X_train.astype("float") / 256
X_test  = X_test.astype("float")  / 256
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (674, 256, 256, 3)
X_test shape: (225, 256, 256, 3)


# 일반 CNN 모델 적용

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

# 모델 구조 정의 
model = Sequential()
model.add(Conv2D(32, (3, 3), (1, 1), input_shape=X_train.shape[1:], padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(32, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Conv2D(64, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))
model.add(Conv2D(128, (3, 3), (1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

# 전결합층
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(256, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________

In [11]:
# 모델 구축하기
from tensorflow.keras import metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.CategoricalAccuracy()])

In [12]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience=50)
mc = ModelCheckpoint(base_dir+'CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
histroy = model.fit(X_train, y_train, batch_size=16, epochs=50, callbacks=[es, mc], validation_split=0.15)

Train on 572 samples, validate on 102 samples
Epoch 1/50
572/572 [==============================] - 10s 18ms/sample - loss: 1.1926 - categorical_accuracy: 0.3497 - val_loss: 1.0960 - val_categorical_accuracy: 0.5000
Epoch 2/50
572/572 [==============================] - 2s 4ms/sample - loss: 1.0987 - categorical_accuracy: 0.3741 - val_loss: 1.0945 - val_categorical_accuracy: 0.3627
Epoch 3/50
572/572 [==============================] - 2s 3ms/sample - loss: 1.0493 - categorical_accuracy: 0.4528 - val_loss: 1.0346 - val_categorical_accuracy: 0.4118
Epoch 4/50
572/572 [==============================] - 2s 3ms/sample - loss: 0.9635 - categorical_accuracy: 0.5490 - val_loss: 1.1447 - val_categorical_accuracy: 0.3922
Epoch 5/50
572/572 [==============================] - 2s 3ms/sample - loss: 0.9044 - categorical_accuracy: 0.5979 - val_loss: 0.8154 - val_categorical_accuracy: 0.6863
Epoch 6/50
572/572 [==============================] - 2s 3ms/sample - loss: 0.8239 - categorical_accuracy: 0.648

In [13]:
# 모델 평가하기 
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')
score = model.evaluate(X_test, y_test)
print('loss=', score[0])        # loss
print('accuracy=', score[1])    # acc

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
225/225 [==============================] - 1s 3ms/sample - loss: 0.2659 - categorical_accuracy: 0.9200
loss= 0.2659363608251119
accuracy= 0.92


In [14]:
from tensorflow.keras.models import load_model
model = load_model(base_dir+'CNN_best_model.h5')

for category in categories:
    for i in range(1,4):
        test_image = base_dir+category+'_test_'+str(i)+'.jpg'
        
        # 이미지 resize
        img = Image.open(test_image)
        img = img.convert("RGB")
        img = img.resize((256,256))
        data = np.asarray(img)
        X = np.array(data)
        X = X.astype("float") / 256
        X = X.reshape(-1, 256,256,3)
        
        # 예측
        pred = model.predict(X)
        print(pred)
        result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
        print(test_image, ':',categories[result[0]])
        print()

[[0.99270767 0.00410783 0.00318456]]
./datasets/pedestrian_test_1.jpg : datasets/pedestrian

[[9.9999368e-01 5.9614104e-06 3.1054847e-07]]
./datasets/pedestrian_test_2.jpg : datasets/pedestrian

[[9.9855453e-01 1.3640116e-03 8.1470644e-05]]
./datasets/pedestrian_test_3.jpg : datasets/pedestrian

[[9.9974126e-01 2.3130572e-04 2.7371520e-05]]
./datasets/sitter_test_1.jpg : datasets/pedestrian

[[1.2961805e-03 9.9870300e-01 8.0635670e-07]]
./datasets/sitter_test_2.jpg : datasets/sitter

[[1.1848636e-02 9.8815084e-01 5.9968841e-07]]
./datasets/sitter_test_3.jpg : datasets/sitter

[[1.5436917e-06 2.1852651e-05 9.9997663e-01]]
./datasets/taxier_test_1.jpg : datasets/taxier

[[4.3292826e-07 5.7162555e-07 9.9999905e-01]]
./datasets/taxier_test_2.jpg : datasets/taxier

[[1.5145622e-05 1.5135946e-05 9.9996972e-01]]
./datasets/taxier_test_3.jpg : datasets/taxier

